# Basic imports

In [1]:
import sys
sys.path.append("..")
import trackhhl.toy.simple_generator as toy
import trackhhl.hamiltonians.simple_hamiltonian as hamiltonian
import numpy as np

# Detector

We are going to define a simple detector geometry of 3 parallel infinitely large modules, placed along the $z$-axis

In [2]:
N_MODULES = 3
LX = float("+inf")
LY = float("+inf")
Z_SPACING = 1.0

detector = toy.SimpleDetectorGeometry(
    module_id=list(range(N_MODULES)),
    lx=[LX]*N_MODULES,
    ly=[LY]*N_MODULES,
    z=[i+Z_SPACING for i in range(N_MODULES)]
)

detector

SimpleDetectorGeometry(module_id=[0, 1, 2], lx=[inf, inf, inf], ly=[inf, inf, inf], z=[1.0, 2.0, 3.0])

# Particle generator

Now we define a simple particle generator that will fire particles flying in straight lines through the our detector.

In [3]:
generator = toy.SimpleGenerator(
    detector_geometry=detector,
    theta_max=np.pi/6
)

Let's generate a simple event

In [4]:
N_PARTICLES = 3
event = generator.generate_event(N_PARTICLES)
event.hits

[Hit(hit_id=0, x=np.float64(-0.4614615823703159), y=np.float64(-0.34023605252461026), z=1.0, module_id=0, track_id=0),
 Hit(hit_id=3, x=np.float64(0.22959853549335138), y=np.float64(0.07388867215047602), z=1.0, module_id=0, track_id=1),
 Hit(hit_id=6, x=np.float64(0.16531675682551708), y=np.float64(0.15659948343374777), z=1.0, module_id=0, track_id=2),
 Hit(hit_id=1, x=np.float64(-0.9229231647406319), y=np.float64(-0.6804721050492205), z=2.0, module_id=1, track_id=0),
 Hit(hit_id=4, x=np.float64(0.45919707098670276), y=np.float64(0.14777734430095205), z=2.0, module_id=1, track_id=1),
 Hit(hit_id=7, x=np.float64(0.33063351365103416), y=np.float64(0.31319896686749554), z=2.0, module_id=1, track_id=2),
 Hit(hit_id=2, x=np.float64(-1.3843847471109478), y=np.float64(-1.0207081575738308), z=3.0, module_id=2, track_id=0),
 Hit(hit_id=5, x=np.float64(0.6887956064800541), y=np.float64(0.22166601645142808), z=3.0, module_id=2, track_id=1),
 Hit(hit_id=8, x=np.float64(0.49595027047655116), y=np.f

# Hamiltonian initialization

Let's initialize the Hamiltonian.

In [5]:
ham = hamiltonian.SimpleHamiltonian(
    epsilon=1e-7,
    gamma=2.0,
    delta=1.0
)

In [6]:
ham.construct_hamiltonian(event=event)

(<18x18 sparse matrix of type '<class 'numpy.float64'>'
 	with 24 stored elements in Compressed Sparse Column format>,
 array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.]))

# Classical solver

Let's solve this event using a classical linear solver (the Conjugate Gradient Descent method)

In [7]:
classical_solution = ham.solve_classicaly()
classical_solution

array([0.5       , 0.33333333, 0.33333333, 0.33333333, 0.5       ,
       0.33333333, 0.33333333, 0.33333333, 0.5       , 0.5       ,
       0.33333333, 0.33333333, 0.33333333, 0.5       , 0.33333333,
       0.33333333, 0.33333333, 0.5       ])

Let's apply a threshold $T = 0.45$

In [8]:
T = .45

In [9]:
discretized_classical_solution = (classical_solution > T).astype(int)
discretized_classical_solution

array([1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1])

# HHL solver

Now let's solve the same event using the HHL linear solver provided by Qiskit.

In [10]:
hhl_solution = ham.solve_hhl()
hhl_solution

AttributeError: 'NumPyMatrix' object has no attribute 'power1'

Let's discretize using the same threshold $T=0.45$

In [ ]:
discretized_hhl_solution = (hhl_solution > .45).astype(int)
discretized_classical_solution

array([1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1])

# Comparison

Let's compare the two solutions with the Monte Carlo truth

In [ ]:
print("Classical solution: \t", discretized_classical_solution)
print("HHL solution: \t\t", discretized_classical_solution)
print()
MC_truth = [ 1 if seg.hit_from.track_id == seg.hit_to.track_id else 0 for seg in ham.segments]
print("MC truth: \t\t",np.array(MC_truth))

Classical solution: 	 [1 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 1]
HHL solution: 		 [1 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 1]

MC truth: 		 [1 0 0 0 1 0 0 0 1 1 0 0 0 1 0 0 0 1]
